
<img align="left" src = https://www.linea.org.br/wp-content/themes/LIneA/imagens/logo-header.png width=120 style="padding: 20px"> <br> 
<img align="right" src = https://jupyter.org/assets/homepage/hublogo.svg width=200 style="padding: 20px"> <br> 
<br>
<br>
<br>
<br>

# Minicurso JupyterHub - Acesso a dados do DES e exemplos 

Última verificação: 06/06/23

Andressa Wille e Gabriel Oliveira

## Bibliotecas e Environment

 Para criar um ambiente e instalar as bibliotecas necessárias a execução desse jupyter notebook, é necessário que os seguintes comandos sejam executados no terminal (conforme explicado nos slides):

```console
conda create -p $HOME/.conda/envs/tutorial
conda env list
conda activate tutorial (ou source activate tutorial)
conda install pip numpy astropy scipy pandas sqlalchemy psycopg2 matplotlib seaborn ipykernel
pip install dblinea
python -m ipykernel install --user --name=tutorial
# Refresh page (F5 no teclado não funciona, tem que ser no ícone mesmo).	
```

É comum que a instalação de novos pacotes seja necessária conforme realizamos alguma tarefa científica.
Nesse caso, se já foi feita a criação de um kernel, é necessário que esse seja removido, que o pacote novo seja instalado e que o kernel seja novamente criado.
Podemos fazer isso utilizando os seguintes comandos:

```console
conda deactivate
jupyter kernelspec list
jupyter kernelspec remove tutorial
conda activate tutorial (ou source activate tutorial)
# (instalação do pacote, conda ou pip)
python -m ipykernel install --user --name=tutorial
```

Realizada a configuração do ambiente, podemos prosseguir com a importação dos pacotes/módulos que serão utilizados nesse minicurso.

## Importando Módulos

In [ ]:
import matplotlib.pyplot as plt #Visualização de gráficos
import pandas as pd #Manejamento de Tabelas e dataframes
import numpy as np #Pacote muntifuncional para uso matemático

#Astropy é um módulo com inúmeras ferramentas úteis na vida dos astronomos/astrofísicos
from astropy import units as u
from astropy.coordinates import SkyCoord

#Comandos do JupyterNotebook para agilizar reimportações
%reload_ext autoreload
%autoreload 2

## Acesso ao banco de dados

In [ ]:
from dblinea import DBBase
db = DBBase()
schema = "des_dr2"  
tablename = "coadd_objects"

## Visualizando colunas da tabela

A descrição de cada coluna pode ser encontrada em:
https://des.ncsa.illinois.edu/releases/dr2/dr2-products/dr2-schema

In [ ]:
db.get_table_columns(tablename, schema=schema)

## Realizando Queries

A partir do dblinea é possível realizar queries, utilizando linguagem SQL. Para isso, utilizaremos a função db.fetchall_df

In [ ]:
help(db.fetchall_df)

**Primeira query**

```sql
SELECT coadd_object_id, ra, dec, galactic_l, galactic_b, extended_class_coadd, mag_auto_g,
mag_auto_r, mag_auto_i, mag_auto_z, mag_auto_y, magerr_auto_g, magerr_auto_r, magerr_auto_i, magerr_auto_z, magerr_auto_y 
FROM des_dr2.coadd_objects 
limit 100000
````

In [ ]:
limit = 100000
query = f"SELECT coadd_object_id, ra, dec, galactic_l, galactic_b, extended_class_coadd, mag_auto_g, mag_auto_r, mag_auto_i, mag_auto_z, mag_auto_y, magerr_auto_g, magerr_auto_r, magerr_auto_i, magerr_auto_z, magerr_auto_y FROM des_dr2.coadd_objects limit {limit}"

**Query retangular**

```sql
SELECT coadd_object_id, ra, dec, galactic_l, galactic_b, extended_class_coadd, mag_auto_g,mag_auto_r, mag_auto_i, mag_auto_z, mag_auto_y, magerr_auto_g, magerr_auto_r, magerr_auto_i, magerr_auto_z, magerr_auto_y 
FROM des_dr2.coadd_objects 
WHERE q3c_poly_query(ra, dec, ARRAY[{xlim[0]}, {ylim[1]}, {xlim[0]}, {ylim[0]}, {xlim[1]}, {ylim[0]}, {xlim[1]}, {ylim[1]}])
limit {limit}
```

In [ ]:
xlim = [32,34]
ylim = [-34,-32]
limit = 100000
query = f"SELECT coadd_object_id, ra, dec, galactic_l, galactic_b, extended_class_coadd, mag_auto_g,mag_auto_r, mag_auto_i, mag_auto_z, mag_auto_y, magerr_auto_g, magerr_auto_r, magerr_auto_i, magerr_auto_z, magerr_auto_y FROM des_dr2.coadd_objects WHERE q3c_poly_query(ra, dec, ARRAY[{xlim[0]}, {ylim[1]}, {xlim[0]}, {ylim[0]}, {xlim[1]}, {ylim[0]}, {xlim[1]}, {ylim[1]}]) limit {limit}"

**Query circular**

```sql
SELECT coadd_object_id, ra, dec, galactic_l, galactic_b, extended_class_coadd, mag_auto_g,mag_auto_r, mag_auto_i, mag_auto_z, mag_auto_y, magerr_auto_g, magerr_auto_r, magerr_auto_i, magerr_auto_z, magerr_auto_y 
FROM des_dr2.coadd_objects 
WHERE q3c_radial_query(ra, dec, {x}, {y}, {r}) 
limit {limit}
```

In [ ]:
x,y = 32,-34
r = 2.0
query = f"SELECT coadd_object_id, ra, dec, galactic_l, galactic_b, extended_class_coadd, mag_auto_g,mag_auto_r, mag_auto_i, mag_auto_z, mag_auto_y, magerr_auto_g, magerr_auto_r, magerr_auto_i, magerr_auto_z, magerr_auto_y FROM des_dr2.coadd_objects WHERE q3c_radial_query(ra, dec, {x}, {y}, {r}) limit {limit}"

**Obtendo o dataframe a partir da query**

In [ ]:
dataframe = db.fetchall_df(query)
dataframe

## Exemplos de Utilização

**Parâmetros dos plots**

In [ ]:
plt.rc('text', usetex=True)
plt.rcParams['figure.figsize'] = (7,6)
plt.rcParams['font.size'] = 15

### 1. Distribuição espacial de objetos no céu (densidade)

In [ ]:
ra =  dataframe['ra']
dec =  dataframe['dec']

plt.hexbin(ra, dec, None,  mincnt=1, cmap='viridis', gridsize=[200,100])
plt.xlabel("ra (°)")
plt.ylabel("dec (°)")
plt.colorbar(label="density of points")
plt.grid()
plt.tight_layout()

### 2. Distribuição espacial de objetos no céu (classificação)

In [ ]:
plt.figure(figsize=[12,5], dpi=300)

ra =  dataframe['ra']
dec =  dataframe['dec']

stars = dataframe['extended_class_coadd'] < 2
galaxies = dataframe['extended_class_coadd'] >= 2

plt.subplot(1,2,1)
plt.scatter(ra[galaxies], dec[galaxies], c='black', marker='.', alpha=0.2, label='galaxies')
plt.xlabel("ra (°)")
plt.ylabel("dec (°)")
plt.grid()
plt.legend(frameon = False)

plt.subplot(1,2,2)
plt.scatter(ra[stars], dec[stars], c='yellow', marker='*', alpha=0.2, label='stars')
plt.xlabel("ra (°)")
plt.ylabel("dec (°)")
plt.grid()
plt.legend(frameon = False)

plt.tight_layout()

### 3. Distribuição de magnitudes

In [ ]:
bands = ['g', 'r', 'i', 'z', 'y']
colors = ['blue', 'green', 'red', 'orange', 'purple']
plt.figure()
bins = np.linspace(9, 37, 57)
for i, (band, color) in enumerate(zip(bands,colors)):
    plt.hist(dataframe[f'mag_auto_{band}'], histtype='stepfilled', bins=bins, label=f'mag {band}', alpha = 0.5, edgecolor = "black", color = color)
    plt.xlabel("magnitude")
    plt.ylabel("Counts")
    plt.xlim(9, 37)
    plt.yscale('log')
    plt.legend()
    plt.grid(True)
    plt.show()

### 4. Ajuste de Gaussiana

Utilizaremos uma função muito conhecida em ajustes de curvas, chamada curve_fit, do scipy.optimize

**Importando o módulo/função**

In [ ]:
from scipy.optimize import curve_fit

**Definindo a função Gaussiana**

In [ ]:
def gauss(x, mu, sigma, A):
    return A*np.exp(-(x-mu)**2/2/sigma**2)

**Distribuição de magnitude da banda g**

Neste exemplo básico, vamos ajustar uma curva gaussiana no histograma de magnitudes na banda g.

In [ ]:
plt.figure()
bins = np.linspace(13, 33, 41)
plt.hist(dataframe['mag_auto_g'], fc="moccasin", ec='sandybrown', bins=bins, label='distribuição mag g')
plt.xlabel("magnitude")
plt.ylabel("counts")
plt.xlim(13, 33)
plt.legend(frameon=False, loc='upper left')
plt.grid(False)
plt.tight_layout()

**Ajuste + figura**

In [ ]:
#plot da distribuição
y,x,_=plt.hist(dataframe['mag_auto_g'], fc="moccasin", ec='sandybrown', bins=bins, label='distribuição mag g')

#Ajuste
x=(x[1:]+x[:-1])/2
expected = (25, 5, 16000)
params, cov = curve_fit(gauss, x, y, expected)
sigma=np.sqrt(np.diag(cov))
x_fit = np.linspace(x.min(), x.max(), 500)

#Plot da linha correspondente ao ajuste
plt.plot(x_fit, gauss(x_fit, *params), color='red', lw=2, label='ajuste gaussiana')
plt.legend(frameon=False, loc='upper left')
plt.xlabel("magnitude")
plt.ylabel("counts")
plt.xlim(13, 33)

#Retorno dos parâmetros associados ao ajuste
data = pd.DataFrame(data={'params': params, 'err': sigma}, index=gauss.__code__.co_varnames[1:])
print(data)

Repare nos resultados:
    A é a altura da gaussiana,
    $\mu$ é o centro da gaussiana,
    e $\sigma$ é o desvio.

### 5. Magnitudes vs erros

In [ ]:
bands = ['g', 'r', 'i', 'z', 'y']
for i, band in enumerate(bands):
    plt.figure(figsize=[10,4])
    plt.subplot(121) 
    mag = np.array(dataframe[f'mag_auto_{band}'])
    err = np.array(dataframe[f'magerr_auto_{band}'])
    mask = (mag>14)&(mag<32)&(err<10)
    plt.hexbin(err[mask], mag[mask], None, mincnt=1, cmap='inferno', gridsize=[200,100], bins='log')
    cbar = plt.colorbar(label='density of points')
    plt.ylabel("mag "+band)
    plt.xlabel("err mag "+band)
    plt.grid(True)
    plt.tight_layout()

### 6. Diagramas cor-magnitude

In [ ]:
bands = ['g', 'r', 'i', 'z', 'y']
mag_diff = {}
stars = dataframe['extended_class_coadd'] < 2
for band,_band in zip(bands, bands[1::]):
    plt.figure(figsize=[10,4])
    plt.subplot(121)
    mag_diff = np.array(dataframe[f'mag_auto_{band}'][stars]) - np.array(dataframe[f'mag_auto_{_band}'][stars])
    mag = np.array(dataframe[f'mag_auto_{_band}'][stars])
                   
    mask = (mag>14)&(mag<32)&(mag_diff<10)
    plt.hexbin(mag_diff[mask], mag[mask], None, mincnt=1, cmap='viridis', gridsize=[200,100], bins='log')
    cbar = plt.colorbar(label='density of points')
    plt.ylabel("mag "+band)
    plt.xlabel(f"{band}-{_band}")
    plt.grid(True)
    plt.tight_layout()

### 6. Diagrama cor-cor

In [ ]:
def CCD(mag1, mag2, mag3, maglim):
    bright = (mag1 <= maglim)
    mag1 = mag1[bright]
    mag2 = mag2[bright]
    mag3 = mag3[bright]
    x = mag1-mag2
    y = mag2-mag3
    return x, y

In [ ]:
plt.figure()

x, y = CCD(dataframe[f'mag_auto_g'], dataframe[f'mag_auto_r'], dataframe[f'mag_auto_i'], 24)

plt.scatter(x[galaxies], y[galaxies], c='blue', marker='.', alpha=0.2, label='galaxies')
plt.scatter(x[stars], y[stars], c='yellow', marker='*', alpha=0.2, label='stars')
plt.ylabel("r-i")
plt.xlabel("g-r")
plt.xlim(-1, 2.2)
plt.ylim(-1, 2.2)
plt.grid(True)
plt.legend()
plt.tight_layout()

In [ ]:
plt.figure()

x, y = CCD(dataframe[f'mag_auto_r'], dataframe[f'mag_auto_i'], dataframe[f'mag_auto_z'], 24)

plt.scatter(x[galaxies], y[galaxies], c='blue', marker='.', alpha=0.2, label='galaxies')
plt.scatter(x[stars], y[stars], c='yellow', marker='*', alpha=0.2, label='stars')
plt.xlabel("i-z")
plt.ylabel("r-i")
plt.xlim(-1, 2.2)
plt.ylim(-1, 2.2)
plt.grid(True)
plt.legend()
plt.tight_layout()